<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/otto_v4.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install polars
!pip install 'polars[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import polars as pl
paths = [
         '/content/drive/My Drive/OTTO/sub_rerank_575.csv', # 0.575
         '/content/drive/My Drive/OTTO/sub_pipeline2_576.csv', # 0.576
         '/content/drive/My Drive/OTTO/sub_rerank_577.csv' # 0.577
        #  '/content/drive/My Drive/OTTO/sub_pip_co-visitiation_577.csv', # 0.577
        #  '/content/drive/My Drive/OTTO/sub_pipeline2_577.csv', # 0.577
        ]

In [6]:
def read_sub(path, weight=1): # by default let us assing the weight of 1 to predictions from each submission, this will be akin to a standard vote ensemble
    '''a helper function for loading and preprocessing submissions'''
    return (
        pl.read_csv(path)
            .with_columns(pl.col('labels').str.split(by=' '))
            .with_columns(pl.lit(weight).alias('vote'))
            .explode('labels')
            .rename({'labels': 'aid'})
            .with_columns(pl.col('aid').cast(pl.UInt32)) # we are casting the `aids` to `Int32`! memory management is super important to ensure we don't run out of resources
            .with_columns(pl.col('vote').cast(pl.UInt8))
    )

In [ ]:
subs = [read_sub(path) for path in paths]

In [23]:
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2')
subs.head()

session_type,aid,vote,vote_right,vote_right2
str,u32,u8,u8,u8
"""12899779_click...",59625,1,1,1
"""12899779_click...",1253524,1,1,1
"""12899779_click...",737445,1,1,1
"""12899779_click...",438191,1,1,1
"""12899779_click...",731692,1,1,1


In [24]:
subs_df_null = (subs
    # .fill_null(0)
    .with_columns((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    #.drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)

In [25]:
subs_df_null_sum=subs_df_null.groupby("session_type", maintain_order=True).agg([
    pl.col("aid").count().alias("count"),
    pl.col("vote_sum").sum().alias("sum")
])
subs_df_null_sum.sort(by='sum').reverse()

session_type,count,sum
str,u32,i64
"""14115617_order...",24,66
"""12899785_click...",20,60
"""12899803_click...",20,60
"""12899815_click...",20,60
"""12899816_click...",20,60
"""12899823_click...",20,60
"""12899829_click...",20,60
"""12899839_click...",20,60
"""12899845_click...",20,60


In [21]:
# subs_df_null.filter(
#     (pl.col("session_type") == "14115617_orders"))

session_type,aid,vote,vote_right,vote_right2,vote_sum
str,u32,u8,u8,u8,u8
"""14115617_order...",1043508,1,1,1,3
"""14115617_order...",1043508,1,1,1,3
"""14115617_order...",986164,1,1,1,3
"""14115617_order...",986164,1,1,1,3
"""14115617_order...",1707008,1,1,1,3
"""14115617_order...",1705681,1,1,1,3
"""14115617_order...",1670410,1,1,1,3
"""14115617_order...",1340535,1,1,1,3
"""14115617_order...",1330815,1,1,1,3


In [26]:
subs_df_0 = (subs
    .fill_null(0)
    .with_columns((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)


In [27]:
subs_df_null_sum=subs_df_null.groupby("session_type", maintain_order=True).agg([
    pl.col("aid").count().alias("count"),
    pl.col("vote_sum").sum().alias("sum")
])
subs_df_null_sum.sort(by='sum').reverse()

session_type,count,sum
str,u32,i64
"""14115617_order...",24,66
"""12899785_click...",20,60
"""12899803_click...",20,60
"""12899815_click...",20,60
"""12899816_click...",20,60
"""12899823_click...",20,60
"""12899829_click...",20,60
"""12899839_click...",20,60
"""12899845_click...",20,60


In [12]:
subs

session_type,aid,vote_sum
str,u32,u8
"""14571581_carts...",984794,3
"""14571581_carts...",1072049,3
"""14571581_carts...",1392029,3
"""14571581_carts...",1124107,3
"""14571581_carts...",1571699,3
"""14571581_carts...",636390,3
"""14571581_carts...",1791780,3
"""14571581_carts...",1236674,3
"""14571581_carts...",622489,3


In [11]:
subs_sum=subs.groupby("session_type", maintain_order=True).agg([
    pl.col("aid").count().alias("count"),
    pl.col("vote_sum").sum().alias("sum")
])
subs_sum.sort(by='sum').reverse()

session_type,count,sum
str,u32,i64
"""14115617_order...",24,70
"""12947153_click...",24,65
"""14101924_click...",24,65
"""14104326_click...",24,65
"""14375856_click...",24,65
"""12927203_click...",24,64
"""12948549_click...",23,64
"""12950660_click...",23,64
"""12952040_click...",23,64


In [ ]:
%%time
preds = subs.groupby('session_type').agg([
    pl.col('aid').head(20).alias('labels')
]) 
# preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))

### 3개

In [4]:
subs_df = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2')
subs_df.head()
# subs = sub1.join(sub2, how='outer', on=['session_type', 'aid'])

session_type,aid,vote,vote_right,vote_right2
str,u32,u8,u8,u8
"""12899779_click...",59625,1,1,1
"""12899779_click...",1253524,1,1,1
"""12899779_click...",737445,1,1,1
"""12899779_click...",438191,1,1,1
"""12899779_click...",731692,1,1,1


In [6]:
subs_df_null=subs_df.with_columns(pl.col('aid').cast(pl.UInt32))
subs_df_null=subs_df_null.with_columns((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
subs_df_null=subs_df_null.sort(by='vote_sum').reverse()
subs_df_null

session_type,aid,vote,vote_right,vote_right2,vote_sum
str,u32,u8,u8,u8,u8
"""14571581_carts...",196038,1,1,1,3
"""14571581_carts...",1780093,1,1,1,3
"""14571581_carts...",1497245,1,1,1,3
"""14571581_carts...",174670,1,1,1,3
"""14571581_carts...",1764910,1,1,1,3
"""14571581_carts...",984794,1,1,1,3
"""14571581_carts...",1072049,1,1,1,3
"""14571581_carts...",1392029,1,1,1,3
"""14571581_carts...",978060,1,1,1,3


In [11]:
subs_df_0=subs_df.with_columns(pl.col('aid').cast(pl.UInt32))
subs_df_0=subs_df_0.fill_null(0)
subs_df_0=subs_df_0.with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
subs_df_0=subs_df_0.sort(by='vote_sum').reverse()

In [ ]:
subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)

subs

<ipython-input-8-a028f947c706>:1: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  subs = (subs


session_type,aid,vote_sum
str,u32,u8
"""14571581_carts...",196038,3
"""14571581_carts...",1780093,3
"""14571581_carts...",1497245,3
"""14571581_carts...",174670,3
"""14571581_carts...",1764910,3


In [31]:
subs_df.filter(
    (pl.col("session_type") == "12899779_clicks"))

session_type,aid,vote,vote_right,vote_right2,vote_sum
str,u32,u8,u8,u8,u8
"""12899779_click...",986164,1,1,1,3
"""12899779_click...",108125,1,1,1,3
"""12899779_click...",485256,1,1,1,3
"""12899779_click...",1460571,1,1,1,3
"""12899779_click...",1667087,1,1,1,3
"""12899779_click...",1660089,1,1,1,3
"""12899779_click...",1138236,1,1,1,3
"""12899779_click...",967363,1,1,1,3
"""12899779_click...",742709,1,1,1,3


In [30]:
subs_df.filter(
    (pl.col("session_type") == "14177147_carts"))


session_type,aid,vote,vote_right,vote_right2,vote_sum
str,u32,u8,u8,u8,u8
"""14177147_carts...",292448,1,1,1,3
"""14177147_carts...",1086272,1,1,1,3
"""14177147_carts...",979240,1,1,1,3
"""14177147_carts...",377454,1,1,1,3
"""14177147_carts...",1050678,1,1,1,3
"""14177147_carts...",987854,1,1,1,3
"""14177147_carts...",277779,1,1,1,3
"""14177147_carts...",199494,1,1,1,3
"""14177147_carts...",178845,1,1,1,3


In [18]:
subs_df.groupby("session_type", maintain_order=True).sum().sort(by='vote_sum').reverse()

session_type,aid,vote,vote_right,vote_right2,vote_sum
str,u32,i64,i64,i64,i64
"""12899779_click...",17141384,20,20,20,60
"""12899780_click...",21439036,20,20,20,60
"""12899781_click...",18616549,20,20,20,60
"""12899782_click...",17182481,20,20,20,60
"""12899783_click...",17372513,20,20,20,60
"""12899784_click...",19369063,20,20,20,60
"""12899785_click...",15742754,20,20,20,60
"""12899786_click...",19306528,20,20,20,60
"""12899787_click...",17780557,20,20,20,60


In [7]:
subs_df_null_sum=subs_df_null.groupby("session_type", maintain_order=True).agg([
    pl.col("vote_sum").count().alias("count"),
    pl.col("vote_sum").sum().alias("sum")
])
subs_df_null_sum.sort(by='sum').reverse()

session_type,count,sum
str,u32,i64
"""12899779_click...",20,60
"""12899780_click...",20,60
"""12899781_click...",20,60
"""12899782_click...",20,60
"""12899783_click...",20,60
"""12899784_click...",20,60
"""12899785_click...",20,60
"""12899786_click...",20,60
"""12899787_click...",20,60


In [ ]:
28개 중에서 36/3=12개만 완전히 같음
30개 중에서 30/3=10개만 완전히 같음

In [8]:
4902794/5015409

0.9775461981266134

In [27]:
subs_df_null_sum.filter(
     (pl.col('count') == 20 )
)

session_type,count,sum
str,u32,i64
"""14571581_carts...",20,60
"""14571580_carts...",20,60
"""14571579_carts...",20,60
"""14571578_carts...",20,60
"""14571577_carts...",20,60
"""14571576_carts...",20,60
"""14571575_carts...",20,60
"""14571574_carts...",20,60
"""14571573_carts...",20,60


In [44]:
subs_df_sum.filter(
    pl.col("count").is_between(25,40),
)

session_type,count,sum
str,u32,i64
"""14561260_carts...",26,45
"""14557152_carts...",25,48
"""14551328_carts...",25,48
"""14547494_carts...",25,48
"""14540118_carts...",25,45
"""14529064_carts...",25,51
"""14528321_carts...",27,42
"""14491552_carts...",25,51
"""14446656_carts...",25,42


In [56]:
subs_sum=subs.groupby("session_type", maintain_order=True).agg([
    pl.col("vote_sum").count().alias("count"),
    pl.col("vote_sum").sum().alias("sum")
])
subs_sum.filter(
    (pl.col('sum') < 60) & (pl.col('count') < 20)
)

session_type,count,sum
str,u32,i64


In [51]:
preds=subs.groupby('session_type').agg([
    pl.col('vote_sum')
]) 
# preds = preds.with_column(pl.col('vote_sum').apply(lambda lst: ' '.join([str(aid) for aid in lst])))

In [52]:
preds

session_type,vote_sum
str,list[u8]
"""13054191_click...","[3, 3, ... 3]"
"""13118997_carts...","[3, 3, ... 3]"
"""13539201_click...","[3, 3, ... 3]"
"""12989735_click...","[3, 3, ... 3]"
"""14091739_carts...","[3, 3, ... 3]"
"""13526836_carts...","[3, 3, ... 3]"
"""12907659_order...","[3, 3, ... 3]"
"""14365868_click...","[3, 3, ... 3]"
"""13297946_click...","[3, 3, ... 3]"


In [48]:
preds.describe()

describe,session_type,vote_sum
str,str,str
"""count""","""5015409""","""5015409"""
"""null_count""","""0""","""0"""
"""mean""",null,null
"""std""",null,null
"""min""","""12899779_carts...","""3 3 3 3 3 3 3 ..."
"""max""","""14571581_order...","""3 3 3 3 3 3 3 ..."
"""median""",null,null


In [19]:
%%time
preds = subs.groupby('session_type').agg([
    pl.col('aid').head(20).alias('labels')
]) 
# preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))
preds

CPU times: user 40.4 s, sys: 4.35 s, total: 44.8 s
Wall time: 3.76 s


session_type,labels
str,list[u32]
"""13593586_carts...","[714397, 393566, ... 1428446]"
"""14436455_carts...","[1234721, 1765338, ... 1453584]"
"""13213361_order...","[694920, 666776, ... 214670]"
"""13470726_click...","[559319, 1564042, ... 521668]"
"""13326770_carts...","[122199, 439283, ... 1225597]"
"""14527723_click...","[986164, 108125, ... 1789228]"
"""13367646_click...","[1420240, 1260870, ... 1116949]"
"""13100582_click...","[1812305, 731129, ... 1562233]"
"""13836422_click...","[642804, 1029122, ... 398344]"


In [ ]:
preds.write_csv('submission.csv')

In [ ]:
preds.write_csv('/content/drive/My Drive/OTTO/submission_ens.csv')

### 5개

In [ ]:
subs_df = subs_df.join(subs[3], how='outer', on=['session_type', 'aid'], suffix='_right3').join(subs[4], how='outer', on=['session_type', 'aid'], suffix='_right4')
subs_df.head()

In [17]:
subs_df = (subs_df
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    # .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)


SyntaxError: ignored

In [15]:
subs_df

session_type,aid,vote,vote_right,vote_right2,vote_right3,vote_right4,vote_sum
str,u32,u8,u8,u8,u8,u8,u8
"""14571581_carts...",196038,1,1,1,0,0,3
"""14571581_carts...",1780093,1,1,1,0,0,3
"""14571581_carts...",1497245,1,1,1,1,0,3
"""14571581_carts...",174670,1,1,1,0,1,3
"""14571581_carts...",1764910,1,1,1,1,0,3
"""14571581_carts...",984794,1,1,1,1,1,3
"""14571581_carts...",1072049,1,1,1,1,1,3
"""14571581_carts...",1392029,1,1,1,1,1,3
"""14571581_carts...",978060,1,1,1,1,0,3


In [ ]:
subs_df_null_sum=subs_df_null.groupby("session_type", maintain_order=True).agg([
    pl.col("vote_sum").count().alias("count"),
    pl.col("vote_sum").sum().alias("sum")
])
subs_df_null_sum.sort(by='sum').reverse()

In [8]:
# import polars as pl
# weight=1
# sub1=pl.read_csv('/content/drive/My Drive/OTTO/sub_pip_co-visitiation_577.csv').with_columns(pl.col('labels').str.split(by=' ')).with_columns(pl.lit(weight).alias('vote'))
# sub2=pl.read_csv('/content/drive/My Drive/OTTO/sub_pipeline2_577.csv').with_columns(pl.col('labels').str.split(by=' ')).with_columns(pl.lit(weight).alias('vote'))
# sub3=pl.read_csv('/content/drive/My Drive/OTTO/sub_rerank_577.csv').with_columns(pl.col('labels').str.split(by=' ')).with_columns(pl.lit(weight).alias('vote'))

In [9]:
# sub1=sub1.explode('labels').rename({'labels': 'aid'})
# sub2=sub2.explode('labels').rename({'labels': 'aid'})
# sub3=sub3.explode('labels').rename({'labels': 'aid'})

In [10]:
# sub1=sub1.with_columns(pl.col('aid').cast(pl.UInt32)).with_columns(pl.col('vote').cast(pl.UInt8))
# sub2=sub2.with_columns(pl.col('aid').cast(pl.UInt32)).with_columns(pl.col('vote').cast(pl.UInt8))
# sub3=sub3.with_columns(pl.col('aid').cast(pl.UInt32)).with_columns(pl.col('vote').cast(pl.UInt8))

In [11]:
# sub1.write_csv('/content/drive/My Drive/OTTO/sub1.csv')

In [12]:
# import polars as pl
# sub1=pl.read_csv('/content/drive/My Drive/OTTO/sub1.csv')
# sub2=pl.read_csv('/content/drive/My Drive/OTTO/sub2.csv')
# sub3=pl.read_csv('/content/drive/My Drive/OTTO/sub3.csv')